In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM, Dense

In [8]:
df = pd.read_csv("../stocks/raw_data_candles/AAPL.csv")

In [9]:
df

,time,open,high,low,close,volume,symbol
0,2021-12-13 04:01:00,178.909663,179.226932,178.830346,178.860090,13331,AAPL
1,2021-12-13 04:02:00,178.959236,179.355822,178.899749,179.197188,7595,AAPL
2,2021-12-13 04:03:00,179.147615,179.464883,178.979066,179.365737,10868,AAPL
3,2021-12-13 04:04:00,179.425224,179.435139,179.355822,179.385566,6277,AAPL
4,2021-12-13 04:05:00,179.415310,179.415310,179.256676,179.256676,1367,AAPL
...,...,...,...,...,...,...,...
235151,2023-02-03 19:55:00,154.095401,154.095401,154.095401,154.095401,347,AAPL
235152,2023-02-03 19:56:00,154.095301,154.105286,154.085316,154.105286,1438,AAPL
235153,2023-02-03 19:57:00,154.105286,154.105286,154.095301,154.095301,588,AAPL
235154,2023-02-03 19:59:00,154.115270,154.115270,154.115270,154.115270,555,AAPL


In [10]:
df = df.drop('symbol', axis=1)

In [11]:
df["time"] = pd.to_datetime(df["time"])

In [12]:
df.index = df.pop('time')

In [13]:
X = np.array(df)

In [15]:
X.shape

(235156, 5)

In [16]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'volume']])

In [23]:
X_train = []
y_train = []
for i in range(15, len(scaled_data)):
    X_train.append(scaled_data[i-15:i, :])
    y_train.append(scaled_data[i, 3]) # use closing price as target variable
X_train = np.array(X_train)
y_train = np.array(y_train)

In [24]:
X_train.shape

(235141, 15, 5)

In [25]:
X_test = X_train[-100:]
y_test = y_train[-100:]
X_train = X_train[-500:-100]
y_train = y_train[-500:-100]

In [26]:
X_train.shape, y_train.shape

((400, 15, 5), (400,))

In [27]:
X_test.shape, y_test.shape

((100, 15, 5), (100,))

In [22]:
X

array([[  178.90966324,   179.2269317 ,   178.83034613,   178.86009005,
        13331.        ],
       [  178.95923644,   179.35582201,   178.8997486 ,   179.19718778,
         7595.        ],
       [  179.14761458,   179.46488304,   178.97906572,   179.36573664,
        10868.        ],
       ...,
       [  154.10528564,   154.10528564,   154.09530085,   154.09530085,
          588.        ],
       [  154.11527043,   154.11527043,   154.11527043,   154.11527043,
          555.        ],
       [  154.10528564,   154.1552096 ,   154.09530085,   154.1552096 ,
         3835.        ]])

In [28]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))

In [29]:
X_train.shape, y_train.shape

((400, 15, 5), (400,))

In [31]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 5)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [32]:
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
13/13 [==============================] - 7s 43ms/step - loss: 0.0609
Epoch 2/50
13/13 [==============================] - 1s 40ms/step - loss: 0.0060
Epoch 3/50
13/13 [==============================] - 1s 41ms/step - loss: 0.0016
Epoch 4/50
13/13 [==============================] - 1s 40ms/step - loss: 3.8184e-04
Epoch 5/50
13/13 [==============================] - 1s 42ms/step - loss: 1.4231e-04
Epoch 6/50
13/13 [==============================] - 1s 41ms/step - loss: 9.2765e-05
Epoch 7/50
13/13 [==============================] - 1s 52ms/step - loss: 6.0716e-05
Epoch 8/50
13/13 [==============================] - 1s 41ms/step - loss: 5.4772e-05
Epoch 9/50
13/13 [==============================] - 1s 44ms/step - loss: 5.2122e-05
Epoch 10/50
13/13 [==============================] - 1s 48ms/step - loss: 5.3126e-05
Epoch 11/50
13/13 [==============================] - 1s 57ms/step - loss: 5.2256e-05
Epoch 12/50
13/13 [==============================] - 1s 44ms/step - loss: 5.1682e-05
E

In [33]:
score = model.evaluate(X_test, y_test)
score

4/4 [==============================] - 1s 14ms/step - loss: 6.5937e-06


6.593711532332236e-06

In [34]:
predicted_closing_prices = model.predict(X_test)

4/4 [==============================] - 3s 16ms/step


In [35]:
predicted_closing_prices

array([[0.52154654],
       [0.5213912 ],
       [0.52122724],
       [0.5210726 ],
       [0.52094233],
       [0.5208054 ],
       [0.52068913],
       [0.5205644 ],
       [0.5204699 ],
       [0.5203607 ],
       [0.520306  ],
       [0.5202873 ],
       [0.5203242 ],
       [0.52032906],
       [0.52033424],
       [0.5203548 ],
       [0.52038866],
       [0.5204318 ],
       [0.5205005 ],
       [0.5206066 ],
       [0.5207489 ],
       [0.52090925],
       [0.5210634 ],
       [0.5212216 ],
       [0.5213893 ],
       [0.5215203 ],
       [0.521672  ],
       [0.5217706 ],
       [0.5218829 ],
       [0.5219835 ],
       [0.5220525 ],
       [0.52210706],
       [0.52215606],
       [0.52214587],
       [0.5221428 ],
       [0.52214414],
       [0.5221424 ],
       [0.52218515],
       [0.52219075],
       [0.52219343],
       [0.52225196],
       [0.52228034],
       [0.52230835],
       [0.5223086 ],
       [0.5223092 ],
       [0.5222798 ],
       [0.52224773],
       [0.522

In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])

In [ ]:
test_predictions = model.predict(X_test).flatten()

plt.figure(figsize=(18, 8))
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])

In [ ]:
plt.figure(figsize=(18, 8))

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions', 
            'Training Observations',
            'Validation Predictions', 
            'Validation Observations',
            'Testing Predictions', 
            'Testing Observations'])

# Merging Models

In [ ]:
# LSTM model
lstm_model = Sequential()
# add LSTM layers
...

# Language model
language_model = Sequential()
# add layers
...

# Merge outputs from both models
merged_output = Concatenate()([lstm_model.output, language_model.output])

# Fully connected neural network
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(merged_output.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# compile and train the model
model.compile(optimizer=Adam(), loss='mse')
model.fit(merged_output, y_train, ...)


# Trying to use multiple companies

In [ ]:
df_2 = pd.read_csv("AAPL.csv")
df_3 = pd.read_csv("TSLA.csv")

In [ ]:
df_3

In [ ]:
merged_df = pd.merge(df_2, df_3, on='time')
merged_df.columns

In [ ]:
merged_df.drop(['open_x', 'high_x', 'low_x', 'volume_x', 'symbol_x',
       'open_y', 'high_y', 'low_y', 'volume_y', 'symbol_y'], axis=1, inplace=True)
merged_df

In [ ]:
merged_df = merged_df.dropna(subset=['close_x', 'close_y'])

In [ ]:
merged_df.rename(columns={'close_x': 'Apple', 'close_y': 'Tesla'}, inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df.index = merged_df.pop('time')

In [ ]:
merged_df

In [ ]:
df

In [28]:
# --- SEQUENCE A (Paris)

day_1 = [10, 25, 50]  # OBSERVATION 1 [temperature, speed, pollution]
day_2 = [13, 10, 70]  # OBSERVATION 2 [temperature, speed, pollution]
day_3 = [ 9,  5, 90]  # OBSERVATION 3 [temperature, speed, pollution]
day_4 = [ 7,  0, 95]  # OBSERVATION 4 [temperature, speed, pollution]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = [110] # Pollution at day 5

# --- SEQUENCE B (Berlin)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = [125]

# --- SEQUENCE C (London)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = [30]

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.expand_dims(np.array([y_a, y_b, y_c]).astype(np.float32), axis=-1)

print(X.shape)
print(y.shape)

(3, 4, 3)
(3, 1, 1)


In [32]:
sq_a = pd.DataFrame(sequence_a)
sq_b = pd.DataFrame(sequence_b)
sq_c = pd.DataFrame(sequence_c)

In [34]:
sqs = [sq_a, sq_b, sq_c]

df_concat = pd.DataFrame(index=sq_a.index, columns=df1.columns)

# Loop through the columns and rows of the new dataframe
for col in df_concat.columns:
    for row in df_concat.index:
        # Initialize an empty list to store the values from the original dataframes
        values = []
        # Loop through the original dataframes and append the value to the list
        for df in sqs:
            values.append(df.loc[row, col])
        # Set the value in the new dataframe to the list of values
        df_concat.loc[row, col] = values

# Check the resulting dataframe
print(df_concat.head())

KeyError: 'open'

In [37]:
np.array(sq_a)

array([[10, 25, 50],
       [13, 10, 70],
       [ 9,  5, 90],
       [ 7,  0, 95]])